# Hidden Markov Models 

#### February 14, 2022

How to segment a sequence into sub-sequences according to their statistical properties

Note: here sequence has a very general meaning, it could be any (one-dimensional) ordered set.

https://www.digitalvidya.com/blog/inroduction-to-hidden-markov-models-using-python/

## What is a Markov model? A random process where the future depends only on the present

Examples: a series of coin flips. (trivial: no dependence on current state).

We can imagine a genome sequence where the probability of the next nucleotide depends on the current one.

## What is a Hidden Markov Model? there is an underlying markov model.  but we don't get to see it.  instead, we observe correlates of that "hidden" state

In [8]:
import numpy as np
import pandas as pd
import networkx as nx                         # networkx

import networkx.drawing.nx_pydot as pydot     # pydot?

import matplotlib.pyplot as plt
from pprint import pprint                  # nice printing of arrays
##matplotlib inline

In [2]:
pip install pydot


The following command must be run outside of the IPython shell:

    $ pip install pydot

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


## Aim: partition a sequence into (non-overlapping) sub-sequences of several different types.

Each "type" of sequence has different characteristics, which are evident only by considering local context.

Example:
* helix/beta-strand/loop
* conserved/non-conserved
* exon-coding/exon-non-coding/intron/intergenic


## Could try windowed averages

* but note that we will need to pick an (arbitrary) window size, and if a true instance is shorter than the window we might miss it.  It will also be difficult to identify the boundaries between the different regimes

# The occasionally cheating casino - a classic example of a Hidden Markov Model

* Fair coin: p(H) = 0.5, p(T) = 0.5
* Weighted coin: p(H) = 0.75, p(T) = 0.25

Given a series of observations, can we infer whether the data was produced by a fair or a weighted coin?

What if the data were produced by a gambler who was mostly using a fair coin, but occasionally (sneakily!) substituting a weighted coin?

In [9]:
## Create *Observable* state space and initial state probabilities
states = ['H', 'T']                     # these are the observable states of the system
pi = [0.5, 0.5]
state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())

H    0.5
T    0.5
Name: states, dtype: float64
1.0


In [11]:
# create Hidden state space and initial state probabilities
hidden_states = ['Fair', 'Load']
pi = [0.5, 0.5]

print('\n')
state_space = pd.Series(pi, index=hidden_states, name='states')
print("(Hidden) state space:") 
print(state_space, "\n")

print('Sum of hidden state prior probabilities:')
print(state_space.sum())



(Hidden) state space:
Fair    0.5
Load    0.5
Name: states, dtype: float64 

Sum of hidden state prior probabilities:
1.0


In [12]:
# Create transition matrix between hidden states, a (or alpha)

a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.9, 0.1]
a_df.loc[hidden_states[1]] = [0.1, 0.9]

print("transition matrix between hidden states (DataFrame form):")
print(a_df, "\n")

print("transition matrix between hidden states (matrix form):")

a = a_df.values
print(a,"\n")

print("Shape:",a.shape,"\n")

print("Sums:")
print(a_df.sum(axis=1))

transition matrix between hidden states (DataFrame form):
     Fair Load
Fair  0.9  0.1
Load  0.1  0.9 

transition matrix between hidden states (matrix form):
[[0.9 0.1]
 [0.1 0.9]] 

Shape: (2, 2) 

Sums:
Fair    1.0
Load    1.0
dtype: float64


In [13]:
# Create matrix of observation (emission) probabilities, b or beta

observable_states = states

b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.5, 0.5]
b_df.loc[hidden_states[1]] = [0.75, 0.25]

print("Emission of observed state Oj when system is in hidden state Si")
print(b_df,"\n")

b = b_df.values
print("matrix form:")
print(b,"\n")

print("shape:",b.shape,"\n")
print('Sums (across)')
print(b_df.sum(axis=1))

Emission of observed state Oj when system is in hidden state Si
         H     T
Fair   0.5   0.5
Load  0.75  0.25 

matrix form:
[[0.5 0.5]
 [0.75 0.25]] 

shape: (2, 2) 

Sums (across)
Fair    1.0
Load    1.0
dtype: float64


In [14]:
# create a function that maps transition probability dataframe
# to markov edges and weights

def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

In [15]:
# create graph edges and weights

hide_edges_wts = _get_markov_edges(a_df)
print("hidden edge weights:")
pprint(hide_edges_wts)
print("\n")

emit_edges_wts = _get_markov_edges(b_df)
print("emit edge weights:")
pprint(emit_edges_wts)
print("\n")

# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(hidden_states)
print('Nodes:')
print(G.nodes(),"\n")

# edges represent hidden probabilities
for k, v in hide_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

# edges represent emission probabilities
for k, v in emit_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    
print('Edges:')
pprint(G.edges(data=True))    

hidden edge weights:
{('Fair', 'Fair'): 0.9,
 ('Fair', 'Load'): 0.1,
 ('Load', 'Fair'): 0.1,
 ('Load', 'Load'): 0.9}


emit edge weights:
{('Fair', 'H'): 0.5,
 ('Fair', 'T'): 0.5,
 ('Load', 'H'): 0.75,
 ('Load', 'T'): 0.25}


Nodes:
['Fair', 'Load'] 

Edges:
OutMultiEdgeDataView([('Fair', 'Fair', {'weight': 0.9, 'label': 0.9}), ('Fair', 'Load', {'weight': 0.1, 'label': 0.1}), ('Fair', 'H', {'weight': 0.5, 'label': 0.5}), ('Fair', 'T', {'weight': 0.5, 'label': 0.5}), ('Load', 'Fair', {'weight': 0.1, 'label': 0.1}), ('Load', 'Load', {'weight': 0.9, 'label': 0.9}), ('Load', 'H', {'weight': 0.75, 'label': 0.75}), ('Load', 'T', {'weight': 0.25, 'label': 0.25})])


In [16]:
# measure a series of coin flips
# observations are encoded numerically
obs_map = {'H':0, 'T':1}
obs = np.random.randint(0,2,100)   #100 random integers in range [0:2] (so excluding 2 itself)

print(obs)

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

# print( pd.DataFrame(np.column_stack([obs, obs_seq]), 
#                columns=['Obs_code', 'Obs_seq']) )

print(obs_seq)

[1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 1 1
 1 1 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1
 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 1 0 1]
['T', 'H', 'H', 'T', 'T', 'T', 'H', 'H', 'T', 'T', 'H', 'H', 'T', 'T', 'H', 'T', 'T', 'T', 'H', 'T', 'H', 'H', 'T', 'H', 'T', 'H', 'T', 'T', 'H', 'H', 'H', 'T', 'H', 'H', 'H', 'T', 'T', 'T', 'T', 'H', 'H', 'H', 'H', 'H', 'T', 'T', 'T', 'T', 'T', 'T', 'H', 'T', 'H', 'H', 'H', 'T', 'H', 'H', 'H', 'T', 'T', 'T', 'H', 'H', 'H', 'H', 'T', 'H', 'T', 'H', 'H', 'H', 'H', 'T', 'T', 'H', 'H', 'H', 'H', 'H', 'T', 'H', 'T', 'T', 'T', 'H', 'H', 'T', 'H', 'T', 'H', 'H', 'T', 'T', 'H', 'T', 'H', 'T', 'H', 'T']


## viterbi algorithm

find the series of *hidden* states that maximize the probability of producing the *emitted* states

In [17]:
def viterbi(pi,a,b,obs):

	nStates = np.shape(b)[0]
	T = np.shape(obs)[0]

	path = np.zeros(T)
	delta = np.zeros((nStates,T))
	phi = np.zeros((nStates,T))

	delta[:,0] = pi * b[:,obs[0]]
	phi[:,0] = 0

	for t in range(1,T):
		for s in range(nStates):
			delta[s,t] = np.max(delta[:,t-1]*a[:,s])*b[s,obs[t]]
			phi[s,t] = np.argmax(delta[:,t-1]*a[:,s])

	path[T-1] = np.argmax(delta[:,T-1])
	for t in range(T-2,-1,-1):
		#path[t] = phi[int(path[t+1]): int(t+1) , int(t+1)]
		path[t] = phi[int(path[t+1]) , int(t+1)]

	return path,delta, phi

In [18]:
# run viterbi algorithm on our coin data
path, delta, phi = viterbi(pi, a, b, obs)
print('\n')
print('single best state path: ', path)
#print('delta:\n', delta)
#print('phi:\n', phi)

state_map = {0:'Fair', 1:'Load'}
state_path = [state_map[v] for v in path]


result = (pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))
 
print(np.array(result.Best_Path))



single best state path:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
['Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'Fair'
 'Fair' 'Fair' 'Fair' 'Fair'

In [19]:
# now try a different sequence
obs = np.random.randint(0,2,100)   #100 random integers in range [0:2] (so excluding 2 itself)

obs[30:40] = 10*[0]

In [20]:
print(10*[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
pprint(obs)

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0])


In [45]:
# run viterbi algorithm (defined below)
path, delta, phi = viterbi(pi, a, b, obs)
print('\n')
print('single best state path: ', path)
# print('delta:\n', delta)
print('phi:\n', phi)

state_map = {0:'Fair', 1:'Load'}
state_path = [state_map[v] for v in path]


result = (pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))
 
output = np.array(result.Best_Path)
print(output)



single best state path:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
phi:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1.
  1. 0. 1. 1.]]
['Fair' 'Fair' 'Fair' 'Fair' 'Fair' 'F

# CpG islands *******************************

### Origin of CpG (CG) islands

The CG island is a short stretch of DNA in which the frequency of the CG sequence is higher than other regions. It is also called the CpG island, where "p" simply indicates that "C" and "G" are connected by a phosphodiester bond.

CpG islands are often located around the promoters of housekeeping genes (which are essential for general cell functions) or other genes frequently expressed in a cell. At these locations, the CG sequence is not methylated. By contrast, the CG sequences in inactive genes are usually methylated to suppress their expression. The methylated cytosine may be converted to thymine by accidental deamination. Unlike the cytosine to uracil mutation which is efficiently repaired, the cytosine to thymine mutation can be corrected only by the mismatch repair which is very inefficient. Hence, over evolutionary time scales, the methylated CG sequence will be converted to the TG sequence. This explains the deficiency of the CG sequence in inactive genes.

http://www.web-books.com/MoBio/Free/Ch7F2.htm



## Create *Observable* state space and initial state probabilities

In [15]:
states = ['A', 'T', 'C', 'G']                     # these are the observable states of the system
pi = [0.25, 0.2, 0.35, 0.2]                       # this is the chance that the system starts in each state

state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())

A    0.25
T    0.20
C    0.35
G    0.20
Name: states, dtype: float64
1.0


## NOTE: MAKE SURE THAT ROWS AND COLUMNS ARE AS EXPECTED.

## Create transition matrix: 
* the probability that the next site is in state T given that the system is currently in state S
* matrix is size (M x M) where M is number of states (here, M=2)

In [16]:
# data frame (easier to read)
q_df = pd.DataFrame(columns=states, index=states)
#q_df.loc[states[0]] = [0.9, 0.1]     # Fair -> Fair = 90%, Fair -> Biased = 10%
#q_df.loc[states[1]] = [0.3, 0.7]     # Biased -> Fair = 30%, Biased -> Biased = 70%

q_df = pd.DataFrame(columns=states, index=states)
q_df.loc[states[0]] = [0.4, 0.2, 0.2, 0.2]
q_df.loc[states[1]] = [0.2, 0.45, 0.1, 0.25]
q_df.loc[states[2]] = [0.25, 0.25, .3, 0.2]
q_df.loc[states[3]] = [0.15, 0.25, .3, 0.3]

print("Data Frame:\n",q_df,"\n")

# make an array (suitable for use with HMM codes)
q = q_df.values

print("Array:\n",q,"\n")

print('shape = ',q.shape,"\n")

print("Note: the probabilities of going from state Oi to any other state has to add up to 100%")

print(q_df.sum(axis=1))

Data Frame:
       A     T    C     G
A   0.4   0.2  0.2   0.2
T   0.2  0.45  0.1  0.25
C  0.25  0.25  0.3   0.2
G  0.15  0.25  0.3   0.3 

Array:
 [[0.4 0.2 0.2 0.2]
 [0.2 0.45 0.1 0.25]
 [0.25 0.25 0.3 0.2]
 [0.15 0.25 0.3 0.3]] 

shape =  (4, 4) 

Note: the probabilities of going from state Oi to any other state has to add up to 100%
A    1.0
T    1.0
C    1.0
G    1.0
dtype: float64


# now make a graph: 

## First create a function that maps transition probability dataframe to Markov edges and weights

In [17]:
def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

edges_wts = _get_markov_edges(q_df)
pprint(edges_wts)

{('A', 'A'): 0.4,
 ('A', 'C'): 0.2,
 ('A', 'G'): 0.2,
 ('A', 'T'): 0.2,
 ('C', 'A'): 0.25,
 ('C', 'C'): 0.3,
 ('C', 'G'): 0.2,
 ('C', 'T'): 0.25,
 ('G', 'A'): 0.15,
 ('G', 'C'): 0.3,
 ('G', 'G'): 0.3,
 ('G', 'T'): 0.25,
 ('T', 'A'): 0.2,
 ('T', 'C'): 0.1,
 ('T', 'G'): 0.25,
 ('T', 'T'): 0.45}


## Use this function to create a graph object (networkx)

In [18]:
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(states)

print('Nodes:\n')
print(G.nodes())
print('\n')

# edges represent transition probabilities
for k, v in edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
print('Edges:')
pprint(G.edges(data=True))


Nodes:

['A', 'T', 'C', 'G']


Edges:
OutMultiEdgeDataView([('A', 'A', {'weight': 0.4, 'label': 0.4}), ('A', 'T', {'weight': 0.2, 'label': 0.2}), ('A', 'C', {'weight': 0.2, 'label': 0.2}), ('A', 'G', {'weight': 0.2, 'label': 0.2}), ('T', 'A', {'weight': 0.2, 'label': 0.2}), ('T', 'T', {'weight': 0.45, 'label': 0.45}), ('T', 'C', {'weight': 0.1, 'label': 0.1}), ('T', 'G', {'weight': 0.25, 'label': 0.25}), ('C', 'A', {'weight': 0.25, 'label': 0.25}), ('C', 'T', {'weight': 0.25, 'label': 0.25}), ('C', 'C', {'weight': 0.3, 'label': 0.3}), ('C', 'G', {'weight': 0.2, 'label': 0.2}), ('G', 'A', {'weight': 0.15, 'label': 0.15}), ('G', 'T', {'weight': 0.25, 'label': 0.25}), ('G', 'C', {'weight': 0.3, 'label': 0.3}), ('G', 'G', {'weight': 0.3, 'label': 0.3})])


## create *Hidden* state space and initial state probabilities

In [19]:
hidden_states = ['Std', 'CpG']
pi = [0.5, 0.5]

print('\n')
state_space = pd.Series(pi, index=hidden_states, name='states')
print("(Hidden) state space:") 
print(state_space, "\n")

print('Sum of hidden state prior probabilities:')
print(state_space.sum())



(Hidden) state space:
Std    0.5
CpG    0.5
Name: states, dtype: float64 

Sum of hidden state prior probabilities:
1.0


## Create transition matrix between hidden states, a (or alpha)

= transition probability matrix of changing states given a state matrix is size (M x M) where M is number of states

In [20]:
a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.9, 0.1]
a_df.loc[hidden_states[1]] = [0.1, 0.9]

print("transition matrix between hidden states (DataFrame form):")
print(a_df, "\n")

print("transition matrix between hidden states (matrix form):")

a = a_df.values
print(a,"\n")

print("Shape:",a.shape,"\n")

print("Sums:")
print(a_df.sum(axis=1))

transition matrix between hidden states (DataFrame form):
     Std  CpG
Std  0.9  0.1
CpG  0.1  0.9 

transition matrix between hidden states (matrix form):
[[0.9 0.1]
 [0.1 0.9]] 

Shape: (2, 2) 

Sums:
Std    1.0
CpG    1.0
dtype: float64


## Create matrix of observation (emission) probabilities, b or beta

= observation probabilities given state matrix is size (M x O) where M is number of states and O is number of different possible observations

In [21]:
observable_states = states

b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.25, 0.25, 0.25, 0.25]
b_df.loc[hidden_states[1]] = [0.1, 0.1, 0.4, 0.4]

print("Emission of observed state Oj when system is in hidden state Si")
print(b_df,"\n")

b = b_df.values
print("matrix form:")
print(b,"\n")

print("shape:",b.shape,"\n")
print('Sums (across)')
print(b_df.sum(axis=1))


Emission of observed state Oj when system is in hidden state Si
        A     T     C     G
Std  0.25  0.25  0.25  0.25
CpG   0.1   0.1   0.4   0.4 

matrix form:
[[0.25 0.25 0.25 0.25]
 [0.1 0.1 0.4 0.4]] 

shape: (2, 4) 

Sums (across)
Std    1.0
CpG    1.0
dtype: float64


## create graph edges and weights

In [22]:
hide_edges_wts = _get_markov_edges(a_df)
print("hidden edge weights:")
pprint(hide_edges_wts)
print("\n")

emit_edges_wts = _get_markov_edges(b_df)
print("emit edge weights:")
pprint(emit_edges_wts)
print("\n")

# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(hidden_states)
print('Nodes:')
print(G.nodes(),"\n")

# edges represent hidden probabilities
for k, v in hide_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

# edges represent emission probabilities
for k, v in emit_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    
print('Edges:')
pprint(G.edges(data=True))    

hidden edge weights:
{('CpG', 'CpG'): 0.9,
 ('CpG', 'Std'): 0.1,
 ('Std', 'CpG'): 0.1,
 ('Std', 'Std'): 0.9}


emit edge weights:
{('CpG', 'A'): 0.1,
 ('CpG', 'C'): 0.4,
 ('CpG', 'G'): 0.4,
 ('CpG', 'T'): 0.1,
 ('Std', 'A'): 0.25,
 ('Std', 'C'): 0.25,
 ('Std', 'G'): 0.25,
 ('Std', 'T'): 0.25}


Nodes:
['Std', 'CpG'] 

Edges:
OutMultiEdgeDataView([('Std', 'Std', {'weight': 0.9, 'label': 0.9}), ('Std', 'CpG', {'weight': 0.1, 'label': 0.1}), ('Std', 'A', {'weight': 0.25, 'label': 0.25}), ('Std', 'T', {'weight': 0.25, 'label': 0.25}), ('Std', 'C', {'weight': 0.25, 'label': 0.25}), ('Std', 'G', {'weight': 0.25, 'label': 0.25}), ('CpG', 'Std', {'weight': 0.1, 'label': 0.1}), ('CpG', 'CpG', {'weight': 0.9, 'label': 0.9}), ('CpG', 'A', {'weight': 0.1, 'label': 0.1}), ('CpG', 'T', {'weight': 0.1, 'label': 0.1}), ('CpG', 'C', {'weight': 0.4, 'label': 0.4}), ('CpG', 'G', {'weight': 0.4, 'label': 0.4})])


## We measure a sequence of observable states (e.g., a DNA or protein sequence)

In [23]:
# observations are encoded numerically
obs_map = {'A':0, 'T':1, 'C':2, 'G':3}
obs = np.array([1,1,2,1,0,1,2,1,3,3,2,3,2,3,2,3,2,3,2,0,2,2,0,1,0,1])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

print( pd.DataFrame(np.column_stack([obs, obs_seq]), 
                columns=['Obs_code', 'Obs_seq']) )

   Obs_code Obs_seq
0         1       T
1         1       T
2         2       C
3         1       T
4         0       A
5         1       T
6         2       C
7         1       T
8         3       G
9         3       G
10        2       C
11        3       G
12        2       C
13        3       G
14        2       C
15        3       G
16        2       C
17        3       G
18        2       C
19        0       A
20        2       C
21        2       C
22        0       A
23        1       T
24        0       A
25        1       T


## Viterbi algorithm

In [24]:
def viterbi(pi,a,b,obs):

	nStates = np.shape(b)[0]
	T = np.shape(obs)[0]

	path = np.zeros(T)
	delta = np.zeros((nStates,T))
	phi = np.zeros((nStates,T))

	delta[:,0] = pi * b[:,obs[0]]
	phi[:,0] = 0

	for t in range(1,T):
		for s in range(nStates):
			delta[s,t] = np.max(delta[:,t-1]*a[:,s])*b[s,obs[t]]
			phi[s,t] = np.argmax(delta[:,t-1]*a[:,s])

	path[T-1] = np.argmax(delta[:,T-1])
	for t in range(T-2,-1,-1):
		#path[t] = phi[int(path[t+1]): int(t+1) , int(t+1)]
		path[t] = phi[int(path[t+1]) , int(t+1)]

	return path,delta, phi

## now, we can find the most likely path through the hidden state space given our observations by running the Viterbi algorithm

In [25]:
path, delta, phi = viterbi(pi, a, b, obs)
print('\n')
print('single best state path: ', path)
print('delta:\n', delta)
print('phi:\n', phi)

state_map = {0:'Std', 1:'CpG'}
state_path = [state_map[v] for v in path]


result = (pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))
 
print(result)



single best state path:  [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0.]
delta:
 [[1.25000000e-01 2.81250000e-02 6.32812500e-03 1.42382813e-03
  3.20361328e-04 7.20812988e-05 1.62182922e-05 3.64911575e-06
  8.21051044e-07 1.84736485e-07 4.15657091e-08 9.35228455e-09
  2.10426402e-09 4.73459406e-10 1.06528366e-10 2.39688824e-11
  5.39299854e-12 1.21342467e-12 3.70604038e-13 1.33417454e-13
  3.00189271e-14 6.75425859e-15 1.55618118e-15 3.50140765e-16
  7.87816722e-17 1.77258762e-17]
 [5.00000000e-02 4.50000000e-03 1.62000000e-03 1.45800000e-04
  1.42382813e-05 3.20361328e-06 2.88325195e-06 2.59492676e-07
  1.45964630e-07 5.25472668e-08 1.89170161e-08 6.81012578e-09
  2.45164528e-09 8.82592302e-10 3.17733229e-10 1.14383962e-10
  4.11782264e-11 1.48241615e-11 5.33669814e-12 4.80302833e-13
  1.72909020e-13 6.22472472e-14 5.60225224e-15 5.04202702e-16
  4.53782432e-17 4.08404189e-18]]
phi:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.

## additional material from cited website

In [26]:

# create state space and initial state probabilities

states = ['O1', 'O2', 'O3']
pi = [0.25, 0.4, 0.35]
state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())

# create transition matrix
# equals transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states

q_df = pd.DataFrame(columns=states, index=states)
q_df.loc[states[0]] = [0.4, 0.2, 0.4]
q_df.loc[states[1]] = [0.45, 0.45, 0.1]
q_df.loc[states[2]] = [0.45, 0.25, .3]

print(q_df)

q = q_df.values
print('\n')
print(q, q.shape)
print('\n')
print(q_df.sum(axis=1))

from pprint import pprint 

# create a function that maps transition probability dataframe 
# to markov edges and weights

def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

edges_wts = _get_markov_edges(q_df)
pprint(edges_wts)

# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(states)
print('Nodes:\n')
print(G.nodes())
print('\n')

# edges represent transition probabilities
for k, v in edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
print('Edges:')
pprint(G.edges(data=True))

########################################################################

#pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')   # This is where the error ir
#nx.draw_networkx(G, pos)

# create edge labels for jupyter plot but is not necessary
edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G , pos, edge_labels=edge_labels) #<<<<<<<<<<<
# nx.drawing.nx_pydot.write_dot(G, 'markov.dot')                 #<<<<<<<<<<<
plt.show()

# create state space and initial state probabilities

hidden_states = ['S1', 'S2']
pi = [0.5, 0.5]
print('\n')
state_space = pd.Series(pi, index=hidden_states, name='states')
print(state_space)
print('\n')
print(state_space.sum())

# create hidden transition matrix
# a or alpha 
#   = transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states

a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.7, 0.3]
a_df.loc[hidden_states[1]] = [0.4, 0.6]

print(a_df)

a = a_df.values
print('\n')
print(a)
print(a.shape)
print('\n')
print(a_df.sum(axis=1))

# create matrix of observation (emission) probabilities
# b or beta = observation probabilities given state
# matrix is size (M x O) where M is number of states 
# and O is number of different possible observations

observable_states = states

b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.2, 0.6, 0.2]
b_df.loc[hidden_states[1]] = [0.4, 0.1, 0.5]

print(b_df)

b = b_df.values
print('\n')
print(b)
print(b.shape)
print('\n')
print(b_df.sum(axis=1))

# create graph edges and weights

hide_edges_wts = _get_markov_edges(a_df)
pprint(hide_edges_wts)

emit_edges_wts = _get_markov_edges(b_df)
pprint(emit_edges_wts)

# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(hidden_states)
print('Nodes:\n')
print(G.nodes())
print('\n')

# edges represent hidden probabilities
for k, v in hide_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

# edges represent emission probabilities
for k, v in emit_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    
print('Edges:')
pprint(G.edges(data=True))    

# pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato') #<<<<<<<<<<<
# nx.draw_networkx(G, pos)                                   #<<<<<<<<<<<
# plt.show()                                                 #<<<<<<<<<<<

# create edge labels 
emit_edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G , pos, edge_labels=emit_edge_labels) #<<<<<<<<<
#plt.show()                                                           #<<<<<<<<<
# nx.drawing.nx_pydot.write_dot(G, 'hidden_markov.dot')               #<<<<<<<<<

# observation sequence of dog's behaviors
# observations are encoded numerically

obs_map = {'O1':0, 'O2':1, 'O3':2}
obs = np.array([1,1,2,1,0,1,2,1,0,2,2,0,1,0,1])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

print( pd.DataFrame(np.column_stack([obs, obs_seq]), 
                columns=['Obs_code', 'Obs_seq']) )

def viterbi(pi,a,b,obs):

	nStates = np.shape(b)[0]
	T = np.shape(obs)[0]

	path = np.zeros(T)
	delta = np.zeros((nStates,T))
	phi = np.zeros((nStates,T))

	delta[:,0] = pi * b[:,obs[0]]
	phi[:,0] = 0

	for t in range(1,T):
		for s in range(nStates):
			delta[s,t] = np.max(delta[:,t-1]*a[:,s])*b[s,obs[t]]
			phi[s,t] = np.argmax(delta[:,t-1]*a[:,s])

	path[T-1] = np.argmax(delta[:,T-1])
	for t in range(T-2,-1,-1):
		#path[t] = phi[int(path[t+1]): int(t+1) , int(t+1)]
		path[t] = phi[int(path[t+1]) , int(t+1)]

	return path,delta, phi

path, delta, phi = viterbi(pi, a, b, obs)
print('\n')
print('single best state path: ', path)
print('delta:\n', delta)
print('phi:\n', phi)

state_map = {0:'S1', 1:'S2'}
state_path = [state_map[v] for v in path]


result = (pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))
 
print(np.array(result.Best_Path))

O1    0.25
O2    0.40
O3    0.35
Name: states, dtype: float64
1.0
      O1    O2   O3
O1   0.4   0.2  0.4
O2  0.45  0.45  0.1
O3  0.45  0.25  0.3


[[0.4 0.2 0.4]
 [0.45 0.45 0.1]
 [0.45 0.25 0.3]] (3, 3)


O1    1.0
O2    1.0
O3    1.0
dtype: float64
{('O1', 'O1'): 0.4,
 ('O1', 'O2'): 0.2,
 ('O1', 'O3'): 0.4,
 ('O2', 'O1'): 0.45,
 ('O2', 'O2'): 0.45,
 ('O2', 'O3'): 0.1,
 ('O3', 'O1'): 0.45,
 ('O3', 'O2'): 0.25,
 ('O3', 'O3'): 0.3}
Nodes:

['O1', 'O2', 'O3']


Edges:
OutMultiEdgeDataView([('O1', 'O1', {'weight': 0.4, 'label': 0.4}), ('O1', 'O2', {'weight': 0.2, 'label': 0.2}), ('O1', 'O3', {'weight': 0.4, 'label': 0.4}), ('O2', 'O1', {'weight': 0.45, 'label': 0.45}), ('O2', 'O2', {'weight': 0.45, 'label': 0.45}), ('O2', 'O3', {'weight': 0.1, 'label': 0.1}), ('O3', 'O1', {'weight': 0.45, 'label': 0.45}), ('O3', 'O2', {'weight': 0.25, 'label': 0.25}), ('O3', 'O3', {'weight': 0.3, 'label': 0.3})])


S1    0.5
S2    0.5
Name: states, dtype: float64


1.0
     S1   S2
S1  0.7  0.3
S2  0.4  